# weakref库
类似c++里的 weak_shared ptr

In [1]:
import weakref
class A(object):
    def __init__(self):
        print('new A!')
    def get(self):
        print('cool!')
a = A()
id(a)

new A!


140294239487536

弱引用，并注册销毁时的回调:

## ref

In [2]:
b = weakref.ref(a,lambda x: print('delete a!'))

In [4]:
type(b)

weakref

In [5]:
b().get()

cool!


## proxy

In [6]:
h = weakref.proxy(a,lambda x: print('proxycb delete a!'))

In [7]:
type(h)

weakproxy

In [8]:
h.get()

cool!


## finalize

finalize给机会在gc前再回调1次.

In [9]:
f = weakref.finalize(a,print, 'a will be gc!')

In [10]:
del a

a will be gc!
proxycb delete a!
delete a!


## del的影响

注意proxy/ref weak引用对象的区别

In [11]:
b

<weakref at 0x7f98cc4f74a8; dead>

In [13]:
h

ReferenceError: weakly-referenced object no longer exists

## alive查看变量gc了？

In [14]:
f.alive

False

## finalizer的应用：销毁临时目录

```sh
A more robust alternative can be to define a finalizer which only references the specific functions and objects that it needs, rather than having access to the full state of the object:
```

In [20]:
import tempfile
import shutil
class TempDir:
    def __init__(self):
        #创建临时目录
        self.name = tempfile.mkdtemp()
        self._finalizer = weakref.finalize(self, shutil.rmtree, self.name)

    def remove(self):
        self._finalizer()

    @property
    def removed(self):
        return not self._finalizer.alive

In [21]:
d = TempDir()

In [23]:
d.removed

False

In [24]:
d.name

'/tmp/tmpa0hs4eue'

In [25]:
d.remove()

In [26]:
d.removed

True

## more

weakref还有很多特性: https://docs.python.org/3.7/library/weakref.html